# Gel debonded, two dimensional model

In [52]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
import netgen.geom2d as geom2d
# from netgen.occ import *
import netgen.meshing as ngm
from ngsolve.webgui import Draw

In [53]:
import problems
import numpy as np
import params
import pickle
from time import time

In [77]:
## get problem parameters and geometry
problem = problems.problem2

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G = problem[0]['G']
geom = problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 1
ord = 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional

print("phi0:", phi0)
print("chi:", chi)
print("G:", G)
print("geom:",geom)
print("BC:", BC)
print("N:", N)

phi0: 0.3
chi: 0.45
G: 0.15647831497059816
geom: geo_2D_bonded.pkl
BC: {'dir_cond': 'faces', 'DIR_FACES': 'bottom'}
N: 1000


In [55]:
phi = lambda J: phi0/J
dH = lambda J: (1-1/1000) * phi(J) + np.log(1-phi(J)) + chi * phi(J)**2

gammafun = lambda lamb: -dH(lamb)/lamb # bonded
# gammafun = lambda lamb: -dH(lamb**3)*lamb

lambda_target = 1.49
gamma_target = gammafun(lambda_target)


gamma = Parameter(gamma_target)
print(gamma)
## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    if ".stp" in geom:
        geo = OCCGeometry(geom)
    else:
        geo = pickle.load(open(geom, "rb"))

    mesh = Mesh(geo.GenerateMesh(maxh=h))
    Draw(mesh)
    return mesh


ParameterCF, val = 0.00365064



In [56]:
mesh_old = mesher(geom, h)  
print(mesh_old.GetBoundaries())

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

('bottom', 'right', 'top', 'left')


In [71]:
L = 90      # measured in mm
d = 3
geo = geom2d.SplineGeometry()
p1,p2,p3,p4 = [ geo.AddPoint (x,y) for x,y in [(-0.5*L,0), (0.5*L,0), (0.5*L, d), (-0.5*L,d)] ]
geo.Append(["line",p1,p2],bc="bottom")
geo.Append(["line",p2,p3])
geo.Append(["line",p3,p4])
geo.Append(["line",p4,p1])
mesh = Mesh(geo.GenerateMesh(maxh=h))
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [62]:
def F(u):
    return Id(dim) + Grad(u)
def Norm(vec):
    return InnerProduct(vec, vec)**0.5

def Gel_energy_functional(F):
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    return 0.5*gamma* Trace(F.trans*F ) + H

def Gel_energy_EDP(F): ## |F|^2 + H => gamma F:Gradv + H'*J'
    # ddet(A(t))/dt = det(A(t))*trace(A^-1(t)*Grad (v))
    J = Det(F)
    phi = phi0/J
    dv = Grad(v)
    invF = Inv(F)
    H_prime = -phi/N + log(1-phi) + phi + chi*phi**2
    edp = gamma * InnerProduct(F,dv) + H_prime * J * Trace(invF*dv)
    return edp

In [63]:
## Generate spaces and forms
if BC["dir_cond"] == "faces":
    fes = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fes = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])


u = fes.TrialFunction()
v = fes.TestFunction()
BF = BilinearForm(fes)
F = Id(dim) + Grad(u)

In [64]:
## Assemble forms
def Assemble_Bilinear_Form(BF, F, form):
    if form == "Functional":
        BF += Variation(Gel_energy_functional(F).Compile()*dx)
        return BF
    elif form == "EDP":
        BF += Gel_energy_EDP(F).Compile() * dx
        return BF
BF = Assemble_Bilinear_Form(BF, F, form)

In [72]:
# def Solver_freeswell(BF, gfu, tol=1e-8, maxiter=250, damp = 0.5):
#     """
#     Solves the problem
#     """
#     res = gfu.vec.CreateVector()
#     w = gfu.vec.CreateVector()
#     history = GridFunction(fes, multidim = 0)
#     # here we may need to add another loop
#     conv = 1
#     for iter in range(maxiter):
#         # Prints before the iteration: number of it, residual, energy
#         print("Energy: ", BF.Energy(gfu.vec), "Residual: ", sqrt(abs(InnerProduct(res,res))), "Iteration: ", iter)
#         # Check if energy is not a number
#         if np.isnan(BF.Energy(gfu.vec)):
#             print("Nan")
#             conv = 0
#             break

#         BF.Apply(gfu.vec, res)
#         BF.AssembleLinearization(gfu.vec)
#         inv = BF.mat.Inverse(freedofs = fes.FreeDofs())        
#         w.data = damp * inv * res
#         gfu.vec.data -= w
#         history.AddMultiDimComponent(gfu.vec)
#         if sqrt(abs(InnerProduct(w,res))) < tol:
#             print("Converged")
#             break
#     return gfu, history, conv

In [73]:
def Solver_Bonded(BF, gfu, tol=1e-8, maxiter=250, damp = 0.2):
    res = gfu.vec.CreateVector()
    du  = gfu.vec.CreateVector()
    for loadstep in np.linspace(0.000114552,gamma_target,50):
        gamma.Set(loadstep)
        print(gamma)
        for it in range(maxiter):
            print ("Newton iteration {:3}".format(it),end=", ")
            print ("energy = {:16}".format(BF.Energy(gfu.vec)),"Residual: ", sqrt(abs(InnerProduct(res,res))))
            if np.isnan(BF.Energy(gfu.vec)):
                print("Nan")
                conv = 0
                return gfu, None, conv
            #solve linearized problem:
            BF.Apply (gfu.vec, res)
            BF.AssembleLinearization (gfu.vec)
            inv = BF.mat.Inverse(fes.FreeDofs())
            du.data = damp * inv * res

            #update iteration
            gfu.vec.data -= du
    return gfu, None, 1

In [74]:
# def petsc_solver(fes, BL, gfu):
#     solver = NonLinearSolver(fes = fes,a = BF,solverParameters={"snes_type": "qn",
#                                             "snes_max_it": 2000,
#                                             "snes_monitor": "",
#                                             "snes_rtol": 1e-6,
#                                             "snes_linesearch_type": "basic",
#                                             "snes_linesearch_damping": 0.3,
#                                             "snes_linesearch_max_it": 100})
#     gfu_petsc =solver.solve(gfu)
#     return gfu_petsc

In [76]:
gfu = GridFunction(fes)
gfu.vec[:] = 0
t1 =  time()

## Solve the problem
print("Solving bonded problem")
gfu, history, conv = Solver_Bonded(BF, gfu)


## Plot solution
"""
The following files can be visualized on the viewer.ipynb file
"""
Draw(gfu)
# pickle.dump(gfu, open(f"Sol_Problem{problem[-2]}/gfu.p", "wb"))
# pickle.dump(mesh, open(f"Sol_Problem{problem[-2]}/mesh.p", "wb"))
# vtk = VTKOutput(ma=mesh, coefs=[gfu], names=["u"], filename=f"{name}_{h}", subdivision=0)

Solving bonded problem
ParameterCF, val = 0.000114552

Newton iteration   0, energy = -41.963157161570805 Residual:  nan
Newton iteration   1, energy =              nan Residual:  0.2227007335395819
Nan


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene